In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib import animation
from itertools import combinations

In [94]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib import animation
from itertools import combinations

## need to add: infected period (Particle)
#               infection rate  (Particle)
#               death           (Particle)
#               time(frame)     (Simulation)
#               linear graph    (Simulation)

# recovered(death) -> T/F, infected -> T/F ?



class Particle:

    def __init__(self, x, y, vx, vy, radius=0.01, styles=None, infection = 1):
        # x, y: position of the particle
        # vx, vy: velocity of the particle
        # radius: radius of particle (circle)
        # style: color of the particle
        # infectin: 1 Susceptible 2 Infected 3 Recovered(Dead)
        # 1 - Blue, 2 - Red, 3 - Black

        self.r = np.array((x, y))
        self.v = np.array((vx, vy))
        self.radius = radius

        self.styles = styles
        if not self.styles:
            # Default value of the style
            if infection == 1:
                self.styles = {'edgecolor': 'b', 'facecolor': 'b'}  # blue
            elif infection == 2:
                self.styles = {'edgecolor': 'r', 'facecolor': 'r'}  # red
            elif infection == 3:
                self.styles = {'edgecolor': 'k', 'facecolor': 'k'}  # black
            
        self.infection = infection

    @property
    def x(self):
        return self.r[0]
    @x.setter
    def x(self, value):
        self.r[0] = value
    @property
    def y(self):
        return self.r[1]
    @y.setter
    def y(self, value):
        self.r[1] = value
    @property
    def vx(self):
        return self.v[0]
    @vx.setter
    def vx(self, value):
        self.v[0] = value
    @property
    def vy(self):
        return self.v[1]
    @vy.setter
    def vy(self, value):
        self.v[1] = value
        
    """def infected(self):
        return self.infection
    
    def infected(self, value):
        self.infection = value
"""
    def overlaps(self, other): # collision detection
        # Does the particle collide with the other particle(circle)?
        # if collide, return True

        return np.hypot(*(self.r - other.r)) < self.radius + other.radius
        # np.hypot(x1, x2[, out]): sqrt(x1**2 + x2**2)
        # np.hypot(*(self.v - other.v)): distance between the center
        
        
        ################ This will make simulator slow. Consider other ways###################

    def draw(self, ax):
        # add this particle's circle patch to the Matplotlib Axes ax

        circle = Circle(xy=self.r, radius=self.radius, **self.styles)
        ax.add_patch(circle)
        return circle

    def advance(self, dt):
        # bounce off the walls

        self.r += self.v * dt # the position of the particle after dt

        # Make the Particles bounce off the walls
        if self.x - self.radius < 0: # hit the boundary of the box (left, right)
            self.x = self.radius
            self.vx = -self.vx
            
        if self.x + self.radius > 2:
            self.x = 2 - self.radius
            self.vx = -self.vx
            
        if self.y - self.radius < 0: # hit the boundary of the box (top, bottom)
            self.y = self.radius
            self.vy = -self.vy
            
        if self.y + self.radius > 2:
            self.y = 2 - self.radius
            self.vy = -self.vy

In [99]:
class Simulation:
    # dynamic simulation

    def __init__(self, n, radius=0.01, styles=None, infection = 1, patient = 0):
        # initialize the simulation with number of n particles

        self.init_particles(n, radius, styles, infection)
        self.patient = patient

    def init_particles(self, n, radius, styles, infection):

        try:
            iterator = iter(radius)
            assert n == len(radius)
        except TypeError:
            # r is not iterable: turn it into a generator that returns the same value n times
            
            def r_gen(n, radius):
                for i in range(n):
                    yield radius
            radius = r_gen(n, radius)

        self.n = n
        self.particles = []
        for i, rad in enumerate(radius):
            # find a random initial position for this particle.
            
            while True:
                x, y = rad + (2 - 2 * rad) * np.random.random(2)
                # vr = 0.1 * np.random.random() + 0.05
                vr = 0.2 ############################################## change speed
                vphi = 2 * np.pi * np.random.random()
                vx, vy = vr * np.cos(vphi), vr * np.sin(vphi)
                particle = Particle(x, y, vx, vy, rad, styles, infection[i])
                
                # if overlaps with other particles, make again / if not, add to particles
                for p2 in self.particles:
                    if p2.overlaps(particle):
                        break
                else:
                    self.particles.append(particle)
                    break ##############################################
    def patient():
        return patient

    def handle_collisions(self):
        # collisions between the particles
        # when collide, the velocity of particle changes

        def change_velocities(p1, p2):
            
            m1, m2 = p1.radius**2, p2.radius**2 # consider mass is proportional to volume
            M = m1 + m2
            r1, r2 = p1.r, p2.r
            d = np.linalg.norm(r1 - r2)**2
            v1, v2 = p1.v, p2.v
            u1 = v1 - 2*m2 / M * np.dot(v1-v2, r1-r2) / d * (r1 - r2)
            u2 = v2 - 2*m1 / M * np.dot(v2-v1, r2-r1) / d * (r2 - r1)
            p1.v = u1
            p2.v = u2
            
        # consider every pairs
        pairs = combinations(range(self.n), 2)
        for i,j in pairs:
            if self.particles[i].overlaps(self.particles[j]):
                change_velocities(self.particles[i], self.particles[j])
                
                if self.particles[i].infection == 2 and self.particles[j].infection != 2:  # if i is infected 
                    self.particles[j].infection = 2
                    self.circles[j].set_color('red')
                    
                    # patient +1
                    self.patient += 1
                    
                    # self.circles.append(particle.draw(self.ax))
                    
                elif self.particles[i].infection != 2 and self.particles[j].infection == 2:  # if j is infected
                    self.particles[i].infection = 2   
                    self.circles[i].set_color('red') 
                    self.patient += 1
               

    def advance_animation(self, dt):

        for i, p in enumerate(self.particles):
            p.advance(dt)
            self.circles[i].center = p.r
        self.handle_collisions()
        return self.circles

    def advance(self, dt):
        
        for i, p in enumerate(self.particles):
            p.advance(dt)
        self.handle_collisions()

    def init(self):

        self.circles = []
        for particle in self.particles:
            self.circles.append(particle.draw(self.ax))
        return self.circles

    def animate(self, i):

        self.advance_animation(0.01)
        return self.circles

    def do_animation(self, save=False):

        fig, self.ax = plt.subplots()
        for s in ['top','bottom','left','right']:
            self.ax.spines[s].set_linewidth(2)
        self.ax.set_aspect('equal', 'box')
        self.ax.set_xlim(0, 2)
        self.ax.set_ylim(0, 2)
        self.ax.xaxis.set_ticks([])
        self.ax.yaxis.set_ticks([])
        anim = animation.FuncAnimation(fig, self.animate, init_func=self.init,
                               frames=3000, interval=2, blit=True)
        if save:
            Writer = animation.writers['ffmpeg']
            writer = Writer(fps=100, bitrate=1800)
            anim.save('collision_spread.mp4', writer=writer)
        else:
            plt.show()

In [100]:
import datetime

In [101]:
start = datetime.datetime.now()

if __name__ == '__main__':
    nparticles = 150
    patient = 2
    # radii = np.random.random(nparticles)*0.03+0.02
    radii = np.array([0.02] * nparticles)
    styles = None
    infenction = np.array([1] * (nparticles - patient) + [patient] * patient)
    sim = Simulation(nparticles, radii, styles, infenction, patient)
    sim.do_animation(save = True)
    
end = datetime.datetime.now()

print(end-start)

0:06:14.627371
